# SNCF - OPEN DATA - API TRANSILIEN - "PROCHAINS DEPARTS"

In [1]:
import json
import requests

## Téléchargement des données SNCF

In [2]:
print(f"Downloading SNCF Open Data...")
url = "https://ressources.data.sncf.com/explore/dataset/sncf-lignes-par-gares-idf/download/?format=json&timezone=Europe/Berlin"
dest = "sncf-lignes-par-gares-idf.json"
r = requests.get(url, allow_redirects=False)
open(dest, 'wb+').write(r.content)
print(f"\t`--> file successfully downloaded as ./{dest}")

	`--> file successfully downloaded as ./sncf-lignes-par-gares-idf.json


Extraction des données liées à la ligne retenue (ligne L)

In [3]:
with open('./sncf-lignes-par-gares-idf.json', 'r',  encoding='utf-8') as f:
    sncf_lines_per_stations = json.load(f)

In [4]:
line_l_stations_by_code = {}

In [5]:
for entry in sncf_lines_per_stations:
    is_line_l_station = entry['fields'].get('l',None) is not None
    if is_line_l_station:
        line_l_stations_by_code[int(entry['fields']['code_uic'])] = {
            'label':entry['fields']['libelle_point_arret'],
            'latitude':0.,
            'longitude':0.
        }

Missing stations (bug in data file?)

In [6]:
line_l_stations_by_code[87386318] = {'label':'NANTERRE UNIVERSITÉ', 'latitude':0., 'longitude':0.}

Order by code... (just for fun)

In [7]:
from collections import OrderedDict
line_l_stations_by_code = OrderedDict(sorted(line_l_stations_by_code.items(), key=lambda x: x[0]))

In [8]:
with open('./sncf-gares-et-arrets-transilien-ile-de-france.json', 'r',  encoding='utf-8') as f:
    sncf_stations = json.load(f)

In [9]:
for station in sncf_stations:
    try:
        code_uic = int(station['fields']['code_uic'])
        line_l_stations_by_code[code_uic]['latitude'] = station['geometry']['coordinates'][1]
        line_l_stations_by_code[code_uic]['longitude'] = station['geometry']['coordinates'][0]
    except:
        pass

In [10]:
fullpath = f"./transilien_line_l_stations_by_code.json"
print(f"saving (as json) data to : {fullpath}")         
with open(fullpath, "w+", encoding='utf-8') as f:
    json.dump(line_l_stations_by_code, f)
print('\t`--> done!')

saving (as json) data to : ./transilien_line_l_stations_by_code.json
	`--> done!


In [11]:
fullpath = f"./transilien_line_l_stations_by_code.csv"
print(f"saving (as txt) data to : {fullpath}")   
with open(fullpath, "w+", encoding='utf-8') as f:
    f.write(f"station,label,latitude,longitude\n")
    for sc, sd in line_l_stations_by_code.items():
        f.write(f"{sc},{sd['label']},{sd['latitude']},{sd['longitude']}\n")
print('\t`--> done!')

saving (as txt) data to : ./transilien_line_l_stations_by_code.csv
	`--> done!


## Description de la ligne L

- premier découpage
- utilité à confimer
- structure de données à revoir

![GitHub Logo](./line-l.png)

In [12]:
# COMMON PART OF [L1, L2, L4]
l000 = [
    'PARIS SAINT-LAZARE (GARE SAINT-LAZARE)',
    'PONT CARDINET',
    'CLICHY LEVALLOIS',
    'ASNIERES SUR SEINE',
    'BECON LES BRUYERES'
]
# COMMON PART OF [L1, L2]
l100 = [*l000]
l100.extend([
    'COURBEVOIE',
    'LA DEFENSE', 
    'PUTEAUX', 
    'SURESNES MONT VALERIEN', 
    "LE VAL D'OR", 
    'SAINT-CLOUD'
])
# LINE L1
L1 = [*l100]
L1.extend([
    "SEVRES VILLE D'AVRAY", 
    'CHAVILLE RIVE DROITE', 
    'VIROFLAY RIVE DROITE', 
    'MONTREUIL', 
    'VERSAILLES RIVE DROITE'])
# LINE L2 
L2 = [*l100]
L2.extend([
    'GARCHES MARNES LA COQUETTE', 
    'VAUCRESSON', 
    'LA CELLE SAINT-CLOUD', 
    'BOUGIVAL', 'LOUVECIENNES', 
    'MARLY LE ROI', 
    "L'ETANG LA VILLE", 
    'SAINT-NOM LA BRETECHE FORET DE MARLY'
])
# LINE L3
L3 = [
    'NOISY LE ROI',
    'SAINT-NOM LA BRETECHE FORET DE MARLY', 
    'MAREIL MARLY', 
    'SAINT-GERMAIN EN LAYE BEL AIR FOURQUEUX', 
    'SAINT-GERMAIN EN LAYE GRANDE CEINTURE'
]
# LINE L4 
L4 = [*l000]
L4.extend([
    'LES VALLEES',
    'LA GARENNE COLOMBES', 
    'NANTERRE UNIVERSITÉ', 
    'HOUILLES CARRIERES SUR SEINE', 
    'SARTROUVILLE', 
    'MAISONS LAFFITTE',
    'ACHERES VILLE',
    "CONFLANS FIN D'OISE",
    'NEUVILLE UNIVERSITÉ',
    'CERGY PREFECTURE',
    'CERGY SAINT-CHRISTOPHE',
    'CERGY LE HAUT'   
])

In [13]:
line_l = {'L1':L1, 'L2':L2, 'L3':L3, 'L4':L4}

## Codes mission des trains de la ligne L
La 1ère lettre du code désigne la destination du train. Généralement, il s’agit de la première lettre du terminus.

- B : Bécon les Bruyères
- D : Saint-Cloud
- F : Maisons-Laffitte
- N : Nanterre Université
- P : Paris Saint-Lazare
- R : Marly le Roi
- S : Saint-Nom la Bretèche
- U : Cergy le Haut
- V : Versailles Rive Droite

La 2ème lettre indique le type de desserte du train.

- I : direct
- A : semi direct
- O : omnibus
- E : autres dessertes
- U : autres dessertes

La 3ème lettre donne une précision sur les gares intermédiaires pour rendre la lettre précédente plus compréhensible.

- A : Asnières sur Seine
- B : Bécon les Bruyères
- C : Clichy Levallois
- L : La Défense ou Les Vallées
- M : Nanterre Université
- P : Pont Cardinet

La 4ème lettre n’a pas de signification particulière, mais elle doit être prononçable.

In [13]:
line_l_missions = {
    'B' : ('BECON LES BRUYERES', ['L1', 'L2', 'L3', 'L4']),
    'D' : ('SAINT-CLOUD', ['L1', 'L2']),
    'F' : ('MAISONS LAFFITTE', ['L4']),
    'N' : ('NANTERRE UNIVERSITÉ', ['L4']),
    'P' : ('PARIS SAINT-LAZARE (GARE SAINT-LAZARE)', ['L1', 'L2', 'L3', 'L4']),
    'R' : ('MARLY LE ROI', ['L2', 'L3']),
    'S' : ('SAINT-NOM LA BRETECHE FORET DE MARLY', ['L2', 'L3']),
    'U' : ('CERGY LE HAUT', ['L4']),
    'V' : ('VERSAILLES RIVE DROITE', ['L1'])
}